In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("~/Desktop/mcc20192/alggen/ags/vnnd/mlptrain.csv", header=None).iloc[0:,:-3].values
data
train_data = data
#train_data, test_data= train_test_split(data, test_size=0.30, random_state=0)
#print(len(train_data),len(test_data))

In [43]:
ys = pd.read_csv("~/Desktop/mcc20192/alggen/ags/vnnd/mlptrain.csv", header=None).iloc[0:,-3:].values
labels = np.zeros(160)
for i in range(0,160):
    if ys[i][0] == 1:
        labels[i] = 0
    if ys[i][1] == 1:
        labels[i] = 1
    if ys[i][2] == 1:
        labels[i] = 2
labels

def correct_labels(predicted):
    counter = 0
    for i in range(0,160):
        if labels[i] == predicted[i]:
            counter += 1
    return counter
from sklearn.cluster import KMeans

In [46]:
c = 0 
for i in range(0,100):
    kmeans = KMeans(n_clusters=3, random_state=i).fit(train_data)
    print(correct_labels(kmeans.predict(train_data)))

159
0
0
53
0
53
0
53
42
0
160
1
0
0
42
0
0
160
160
0
64
54
65
158
2
0
64
53
53
0
53
63
160
0
64
158
42
65
42
160
53
160
42
54
0
0
65
63
158
65
42
53
64
53
65
1
44
53
44
0
160
0
42
44
160
42
54
0
0
0
53
65
65
0
42
159
42
44
160
158
0
42
0
53
42
42
0
0
0
0
0
65
2
0
160
0
42
42
44
53


In [ ]:
[159
0
0
53
0
53
0
53
42
0
160
1
0
0
42
0
0
160
160
0
64
54
65
158
2
0
64
53
53
0
53
63
160
0
64
158
42
65
42
160
53
160
42
54
0
0
65
63
158
65
42
53
64
53
65
1
44
53
44
0
160
0
42
44
160
42
54
0
0
0
53
65
65
0
42
159
42
44
160
158
0
42
0
53
42
42
0
0
0
0
0
65
2
0
160
0
42
42
44
53]

In [5]:
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2], dtype=int32)

In [6]:
correct_labels(kmeans.predict(data))

159

In [21]:
#Implementation of vnnd clustering
nc = 3 # number of clusters
def distance(p,q):
    s = 0.0
    for i in range(0,len(p)):
        s += (p[i] - q[i])**2
    return np.sqrt(s)

# cluster is an array of indices
# x an index 
def dmin(x, cluster): 
    mindist = 13
    for y in cluster:
        if y != x and distance(train_data[x],  train_data[y]) < mindist:
            mindist = distance(train_data[x], train_data[y])
    if mindist < 13:
        return mindist
    else: return 0
    
def dmin_bar(cluster):
    dmin_sum = 0.0
    for x in cluster:
        dmin_sum += dmin(x,cluster)
    return dmin_sum/len(cluster)

def v(cluster):
    v_of_cluster = 0.0
    dmin_bar_of_cluster = dmin_bar(cluster)
    for x in cluster:
        v_of_cluster += (dmin(x,cluster) - dmin_bar_of_cluster)**2
    return v_of_cluster/(len(cluster)-1)
    
def vnnd(clusters):
    v_sum = 0.0
    for C in clusters:
        v_sum += v(C)
    return v_sum        

In [40]:
r1 = np.arange(53)
r2 = np.arange(65)
r3 = np.arange(42)
for i in range(0,65):
    r2[i] = 53 + r2[i]
for i in range(0,42):
    r3[i] = 53 + 65 + r3[i]
    
vnnd(np.array([r1,r2,r3]))

0.026296748844695583

In [8]:
def clustering(individual):
    cluster0 = []
    cluster1 = []
    cluster2 = []
    for i in range(0, len(individual)):
        if individual[i] == 0:
            cluster0.append(i)
        if individual[i] == 1:
            cluster1.append(i)
        if individual[i] == 2:
            cluster2.append(i)
    return np.array([cluster0, cluster1, cluster2])
def fitness(individual):
    return vnnd(clustering(individual))
def populationFitness(population):
    fits = np.zeros(len(population))
    for i in range(0,len(population)):
        fits[i] = fitness(population[i])
    return fits
def orderByFitness(population, fitness):
    for i in range(0, len(population)):
        for j in range(0, len(population)):
            if fitness[i] < fitness[j]:
                swap = fitness[i]
                fitness[i] = fitness[j]
                fitness[j] = swap
                temp = population[i]
                population[i] = population[j]
                population[j] = temp
    return population

In [9]:
import random
W = 160

def startPopulation(N):
    population = np.ndarray(shape=(N,W))
    for i in range(0,N):
        for j in range(0,W):
            population[i][j] = random.randint(0,2)
    return population            
            
def anularCrossover(population):
    new1 = np.zeros(W)
    new2 = np.zeros(W)
    N = len(population)    
    for i in range(0, int(len(population)/2)):
        x = random.randint(0, W-1)
        y = random.randint(0, W-1)
        if y < x: 
            swap = x
            x = y
            y = swap
        for j in range(0, x):
            new1[j] = population[i][j]
            new2[j] = population[N-i-1][j]
        for j in range(x,y): 
            new1[j] = population[N-i-1][j]
            new2[j] = population[i][j]
        for j in range(y,W): 
            new1[j] = population[i][j]
            new2[j] = population[N-i-1][j]
        population[i] = new1
        population[N-i-1] = new2
    return population

def uniformMutation(population):
    for individual in population:
        # 8 is the expeted number of entries to mutate for pm = 0.05
        for m in range(0,8):
            j = random.randint(0,W-1)
            if individual[j] == 0:
                if random.uniform(0,1) < 0.5:
                    individual[j] = 1
                else: 
                    individual[j] = 2
            if individual[j] == 1:
                if random.uniform(0,1) < 0.5:
                    individual[j] = 0
                else: 
                    individual[j] = 2
            else: 
                if random.uniform(0,1) < 0.5:
                    individual[j] = 0
                else: 
                    individual[j] = 1
    return population
    
def EGAvnnd(N, G):
    population = startPopulation(N)
    for i in range(0,G):
        tpopulation = population
        fitness = populationFitness(population)
        tpopulation = orderByFitness(tpopulation, fitness)
        tpopulation = anularCrossover(tpopulation)
        tpopulation = uniformMutation(tpopulation)
        tfitness = populationFitness(tpopulation)
        
        fpopulation = np.concatenate((population,tpopulation))
        ffitness = np.concatenate((fitness,tfitness))
        population = orderByFitness(fpopulation, ffitness)[:N]
    return population[0]

In [41]:
EGAvnnd(10,10)

array([0., 2., 2., 1., 0., 2., 0., 2., 1., 2., 1., 2., 2., 0., 2., 1., 2.,
       0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 2., 0.,
       0., 1., 0., 1., 2., 2., 1., 1., 0., 1., 2., 2., 2., 1., 0., 2., 1.,
       2., 0., 0., 2., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 2.,
       2., 1., 1., 1., 0., 0., 2., 2., 0., 0., 1., 0., 1., 1., 2., 1., 0.,
       2., 1., 0., 2., 1., 2., 0., 1., 2., 0., 2., 2., 1., 1., 1., 1., 2.,
       0., 1., 1., 0., 2., 0., 0., 2., 0., 2., 0., 0., 1., 0., 1., 1., 0.,
       0., 0., 1., 2., 1., 2., 2., 2., 1., 2., 2., 0., 0., 1., 0., 0., 0.,
       0., 2., 2., 1., 2., 0., 1., 2., 2., 0., 1., 0., 1., 2., 2., 2., 0.,
       0., 0., 0., 1., 0., 0., 2.])

In [42]:
best = np.array([0., 2., 2., 1., 0., 2., 0., 2., 1., 2., 1., 2., 2., 0., 2., 1., 2.,
       0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 2., 0.,
       0., 1., 0., 1., 2., 2., 1., 1., 0., 1., 2., 2., 2., 1., 0., 2., 1.,
       2., 0., 0., 2., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 2.,
       2., 1., 1., 1., 0., 0., 2., 2., 0., 0., 1., 0., 1., 1., 2., 1., 0.,
       2., 1., 0., 2., 1., 2., 0., 1., 2., 0., 2., 2., 1., 1., 1., 1., 2.,
       0., 1., 1., 0., 2., 0., 0., 2., 0., 2., 0., 0., 1., 0., 1., 1., 0.,
       0., 0., 1., 2., 1., 2., 2., 2., 1., 2., 2., 0., 0., 1., 0., 0., 0.,
       0., 2., 2., 1., 2., 0., 1., 2., 2., 0., 1., 0., 1., 2., 2., 2., 0.,
       0., 0., 0., 1., 0., 0., 2.])
len(best)
print(correct_labels(best))
clustering(best)

59


array([list([0, 4, 6, 13, 17, 19, 20, 21, 22, 24, 26, 27, 28, 29, 33, 34, 36, 42, 48, 52, 53, 56, 59, 60, 61, 62, 63, 64, 72, 73, 76, 77, 79, 84, 87, 91, 94, 102, 105, 107, 108, 110, 112, 113, 115, 118, 119, 120, 130, 131, 133, 134, 135, 136, 141, 145, 147, 152, 153, 154, 155, 157, 158]),
       list([3, 8, 10, 15, 18, 23, 25, 30, 31, 35, 37, 40, 41, 43, 47, 50, 55, 57, 58, 65, 66, 69, 70, 71, 78, 80, 81, 83, 86, 89, 92, 97, 98, 99, 100, 103, 104, 114, 116, 117, 121, 123, 127, 132, 139, 142, 146, 148, 156]),
       list([1, 2, 5, 7, 9, 11, 12, 14, 16, 32, 38, 39, 44, 45, 46, 49, 51, 54, 67, 68, 74, 75, 82, 85, 88, 90, 93, 95, 96, 101, 106, 109, 111, 122, 124, 125, 126, 128, 129, 137, 138, 140, 143, 144, 149, 150, 151, 159])],
      dtype=object)